## Group 1: Final Project - SimpleRNN on defog
### Sihe Zheng, szheng12@depaul.edu
### YunTzu Yu, yyu54@depaul.edu
### Saruul Enkhtur, senkhtur@depaul.edu
### Lukasz Grzybek, lgrzybek@depaul.edu


#### DePaul University, College of Computing and Digital Media,
#### DSC 672 DATA SCIENCE CAPSTONE


In [ ]:
pip install -q -U keras-tuner


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Normalization
from sklearn.preprocessing import StandardScaler
import keras_tuner as kt
import matplotlib.pyplot as plt

2024-03-14 16:16:53.190538: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-14 16:17:07.265455: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Matplotlib is building the font cache; this may take a moment.


In [ ]:
full_defog = pd.read_csv('full_defog.csv')
full_defog

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Medication,Age,Sex,YearsSinceDx,NFOGQ
0,1000,-0.953630,0.023207,-0.397874,0,0,0,0,61,0,7.0,22
1,1001,-0.964072,0.024408,-0.398529,0,0,0,0,61,0,7.0,22
2,1002,-0.968473,0.028320,-0.387927,0,0,0,0,61,0,7.0,22
3,1003,-0.965339,0.028354,-0.371597,0,0,0,0,61,0,7.0,22
4,1004,-0.954387,0.029757,-0.361989,0,0,0,0,61,0,7.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...
4090525,122967,-0.935958,-0.098663,-0.055577,0,0,0,0,72,0,14.0,15
4090526,122968,-0.941311,-0.122848,-0.083050,0,0,0,0,72,0,14.0,15
4090527,122969,-0.941117,-0.134056,-0.107400,0,0,0,0,72,0,14.0,15
4090528,122970,-0.945181,-0.131040,-0.117320,0,0,0,0,72,0,14.0,15


In [ ]:
array_full_defog = np.array(full_defog)
X_full_defog = array_full_defog[:,1:4]
X_full_defog

array([[-0.95362971,  0.02320705, -0.39787431],
       [-0.96407181,  0.02440839, -0.39852886],
       [-0.96847331,  0.02832031, -0.38792694],
       ...,
       [-0.94111683, -0.13405631, -0.10739978],
       [-0.945181  , -0.13104026, -0.11731993],
       [-0.95248012, -0.11673514, -0.1200763 ]])

In [ ]:
array_full_defog = np.array(full_defog)
y_full_defog = array_full_defog[:,4:7]
y_full_defog

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [ ]:
scaler = StandardScaler()
X_full_defog = scaler.fit_transform(X_full_defog)
X_full_defog = X_full_defog.reshape((X_full_defog.shape[0], 1, X_full_defog.shape[1]))

In [ ]:
X_train_defog, X_test_defog, y_train_defog, y_test_defog = train_test_split(X_full_defog, y_full_defog, test_size=0.20, random_state=0)

In [ ]:
def SimpleRNN_model(hp):
    hp_units = hp.Int('units', min_value=96, max_value=256, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-9, 1e-12])
    #hp_dropoutrate = hp.Float("dropout_rate",min_value=0.10,max_value=0.30,step=0.10)
    #hp_layers = hp.Int("n_layers",min_value=2,max_value=6,step=2)
    hp_activation = hp.Choice("activation", ["relu", "tanh", "sigmoid","softmax"])



    model = keras.Sequential()
    model.add(keras.layers.SimpleRNN(units=hp_units, activation=hp_activation, return_sequences=True, input_shape=(X_train_defog.shape[1], X_train_defog.shape[2])))
    #for i in range(1, hp_layers):
    #model.add(keras.layers.SimpleRNN(units=hp_units, activation=hp_activation,  return_sequences=True))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Conv1D(96,3, activation="relu", padding='same'))
        #if hp.Boolean("dropout"):
            #model.add(keras.layers.Dropout(hp_dropoutrate))

    model.add(keras.layers.Dense(units=3, activation="relu"))



    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='mse',
                  metrics=["accuracy"])
    return model

In [ ]:
tuner1 = kt.Hyperband(SimpleRNN_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     directory = "./untitled_project_2/",
                     project_name = 'proj2')

Reloading Tuner from ./untitled_project_2/proj2/tuner0.json


In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


In [ ]:
tuner1.search(X_train_defog, y_train_defog, epochs=10, validation_split=0.2, callbacks=[stop_early])


best_hps=tuner1.get_best_hyperparameters(num_trials=1)[0]

print(best_hps.get('units'),best_hps.get('learning_rate'))


Trial 30 Complete [01h 14m 24s]
val_accuracy: 0.5309773087501526

Best val_accuracy So Far: 0.8329964280128479
Total elapsed time: 1d 21h 16m 47s
256 1e-12


Best val_accuracy So Far: 0.8329964280128479
Total elapsed time: 1d 21h 16m 47s
256 1e-12

In [ ]:
best_hps.get('units')

256

In [ ]:
best_hps.get('learning_rate')

1e-12

In [ ]:
best_hps.get('activation')

'relu'

In [ ]:
tuner1.results_summary(num_trials=30)

Results summary
Results in ./untitled_project_2/proj2
Showing 30 best trials
Objective(name="val_accuracy", direction="max")

Trial 0020 summary
Hyperparameters:
units: 256
learning_rate: 1e-12
activation: relu
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.8329964280128479

Trial 0024 summary
Hyperparameters:
units: 256
learning_rate: 1e-12
activation: relu
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0020
Score: 0.8326694369316101

Trial 0001 summary
Hyperparameters:
units: 160
learning_rate: 1e-09
activation: softmax
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.7831414341926575

Trial 0012 summary
Hyperparameters:
units: 160
learning_rate: 1e-09
activation: softmax
tuner/epochs: 4
tuner/initial_epoch: 2
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 0001
Score: 0.7780712842941284

Trial 0016 summary
Hyperparameters:
units: 160
learning_rate: 1e-09
activation: softmax
tuner

In [ ]:
all_trials = tuner1.get_best_hyperparameters(num_trials=30)

In [ ]:
srnn_model = tuner1.hypermodel.build(best_hps)
history = srnn_model.fit(X_train_defog, y_train_defog, epochs=10, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1

print(best_epoch,)


Epoch 1/10
81811/81811 [==============================] - 593s 7ms/step - loss: 0.4011 - accuracy: 0.0838 - val_loss: 0.4443 - val_accuracy: 0.0786
Epoch 2/10
81811/81811 [==============================] - 589s 7ms/step - loss: 0.4011 - accuracy: 0.0838 - val_loss: 0.4320 - val_accuracy: 0.0810
Epoch 3/10
81811/81811 [==============================] - 605s 7ms/step - loss: 0.4011 - accuracy: 0.0838 - val_loss: 0.4105 - val_accuracy: 0.0751
Epoch 4/10
81811/81811 [==============================] - 600s 7ms/step - loss: 0.4011 - accuracy: 0.0838 - val_loss: 0.4250 - val_accuracy: 0.0787
Epoch 5/10
81811/81811 [==============================] - 602s 7ms/step - loss: 0.4011 - accuracy: 0.0838 - val_loss: 0.4151 - val_accuracy: 0.0797
Epoch 6/10
81811/81811 [==============================] - 617s 8ms/step - loss: 0.4012 - accuracy: 0.0839 - val_loss: 0.4240 - val_accuracy: 0.0800
Epoch 7/10
81811/81811 [==============================] - 606s 7ms/step - loss: 0.4012 - accuracy: 0.0837 - val_

In [ ]:
hypermodel = tuner1.hypermodel.build(best_hps)


hypermodel.fit(X_train_defog, y_train_defog, epochs=2, validation_split=0.2)


Epoch 1/2
81811/81811 [==============================] - 620s 8ms/step - loss: 0.1179 - accuracy: 0.2830 - val_loss: 0.1283 - val_accuracy: 0.2914
Epoch 2/2
81811/81811 [==============================] - 606s 7ms/step - loss: 0.1180 - accuracy: 0.2831 - val_loss: 0.1277 - val_accuracy: 0.2916


In [ ]:
#test set
result = hypermodel.evaluate(X_test_defog, y_test_defog)
print("[loss, accuracy]:", result)


25566/25566 [==============================] - 42s 2ms/step - loss: 0.1288 - accuracy: 0.2904
[loss, accuracy]: [0.12883572280406952, 0.29040083289146423]


# Manual Model

In [ ]:
def SimpleRNN_model(units,lr,activation):

    model = keras.Sequential()
    model.add(keras.layers.SimpleRNN(units=units, activation=activation, return_sequences=True, input_shape=(X_train_defog.shape[1], X_train_defog.shape[2])))

    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Conv1D(96,3, activation="relu", padding='same'))


    model.add(keras.layers.Dense(units=3, activation="relu"))



    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss='mse',
                  metrics=["accuracy"])
    return model

In [ ]:
manual_model = SimpleRNN_model(best_hps.get('units'),best_hps.get('learning_rate'),best_hps.get('activation'))

In [ ]:
manual_model.fit(X_train_defog, y_train_defog,
          epochs=4,
          validation_split=0.2)

Epoch 1/4
81811/81811 [==============================] - 318s 4ms/step - loss: 0.0778 - accuracy: 0.5088 - val_loss: 0.0776 - val_accuracy: 0.5085
Epoch 2/4
81811/81811 [==============================] - 316s 4ms/step - loss: 0.0778 - accuracy: 0.5086 - val_loss: 0.0778 - val_accuracy: 0.5031
Epoch 3/4
81811/81811 [==============================] - 505s 6ms/step - loss: 0.0778 - accuracy: 0.5087 - val_loss: 0.0766 - val_accuracy: 0.5095
Epoch 4/4
81811/81811 [==============================] - 1231s 15ms/step - loss: 0.0777 - accuracy: 0.5088 - val_loss: 0.0797 - val_accuracy: 0.5002


In [ ]:
#test set
result = manual_model.evaluate(X_test_defog, y_test_defog)
print("[loss, accuracy]:", result)

25566/25566 [==============================] - 189s 7ms/step - loss: 0.0798 - accuracy: 0.4997
[loss, accuracy]: [0.07977806776762009, 0.4996972680091858]
